In [1]:
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# imports
import os

import torch, torchvision
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10

torch.__version__, torchvision.__version__
torch.cuda.is_available()
# Install Albumentation library
%pip install albumentations
# Install LR finder
%pip install torchinfo torch_lr_finder
# Install GradCam
%pip install grad-cam

%pip install tqdm


os.environ['KMP_DUPLICATE_LIB_OK']='True'
import math
from collections import OrderedDict
import sys
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR, OneCycleLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch_lr_finder import LRFinder

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import gradio as gr


/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 19.6 MB/s eta 0:00:0000:0100:01
  Created wheel for grad-cam: filename=grad_cam-1.4.8-py3-none-any.whl size=38263 sha256=1b1dae968c2b69836c22454a3a7224675069afdc234545d05b86e9d53dcdd6ff
  Stored in directory: /home/nanekja/.cache/pip/wheels/f8/04/36/94ff3c8a4215826a21946b34c01180817e606989fdf53f7cd6
Successfully built grad-cam
Note: you may need to 

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [ ]:
!git clone https://github.com/nanekja/pytorch_utils

sys.path.append('./pytorch_utils/')
import utils

In [ ]:
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
AVAIL_GPUS

0

In [ ]:
train = datasets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
test = datasets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
mean, std = utils.get_mean_and_std()

In [ ]:
exp = torchvision.datasets.CIFAR10('./data', train=True, download=True)
utils.plot_data(exp,3,8)

In [ ]:
dataiter = iter(transform.train_loader)

images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

# show images
utils.imshow(torchvision.utils.make_grid(images))

In [ ]:
from models import c_resnet

model = c_resnet.Net().to(device)

!pip install torchscan --no-deps
from torchscan import summary

summary(model, (3, 32, 32), receptive_field=True, max_depth=1)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.00001,momentum=0.9 ,weight_decay = 0.0005)

ideal_lr=utils.find_lr(model, optimizer, criterion, transform.train_loader)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=ideal_lr, momentum=0.9,weight_decay=0.0001, nesterov=True)
scheduler = ReduceLROnPlateau(optimizer, patience = 2)

In [ ]:
EPOCHS = 20

net, history = main.fit_model(model, device, transform.train_loader, transform.test_loader, optimizer, criterion, scheduler, EPOCHS)

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
utils.result_graphs(history)

In [ ]:
# Denormalize the data using test mean and std deviation
inv_normalize = transforms.Normalize(
    mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
    std=[1/0.23, 1/0.23, 1/0.23]
)

# Get the misclassified data from test dataset
misclassified_data = utils.get_misclassified_data(model, device, test_loader)

# Plot the misclassified data
utils.display_cifar_misclassified_data(misclassified_data, classes, inv_normalize, number_of_samples=20)

In [ ]:
target_layers = [model.layer4[-1]]
# targets = [ClassifierOutputTarget(7)]
targets = None

utils.display_gradcam_output(misclassified_data, classes, inv_normalize, model, target_layers, targets, number_of_samples=20, transparency=0.70)

In [ ]:


def hello(input1, input2, input3):
    return f'{input1} + {input2} Rohan'

demo = gr.Interface(hello, ["text", gr.Slider(0, 5, step=1), gr.Image(shape=(32, 32))], "text")
demo.launch()